In [ ]:
import pandas as pd 
import os 
from tqdm import tqdm
from Bio import SeqIO


In [ ]:
seq_to_remove = SeqIO.parse(f"{path_DePP_results}/remove_benchmark.fasta", "fasta")
seq_to_remove_list = [record.seq for record in seq_to_remove]

In [ ]:
df_final_results = pd.read_csv(f"{path_DePP_results}/full_benchmark_dataframe.4L.2008.csv", header = 0, sep = ",")
df_final_results = df_final_results[df_final_results["protein_seq"].str.len() >= 200]

df_final_results_unseen = df_final_results[~df_final_results["protein_seq"].isin(seq_to_remove_list)]

In [ ]:
data_dict = {"Deposcope" : df_final_results_unseen["scores_DepolyX"], 
             "PhageDPO" : (df_final_results_unseen["scores_phageDPO"]/100), 
             "DePP" : df_final_results_unseen["scores_DePP"], 
             "Label" : df_final_results_unseen["label"]}

data_color = {"Deposcope" : "#b22234", 
             "PhageDPO" : "#3c3b6e", 
             "DePP" : "#828282"}

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_recall_curve, auc

def plot_multiple_precision_recall_curves(data_dict):
    plt.figure(figsize=(12, 8))
    lw = 2
    sns.set(style="whitegrid")
    for key, value in data_dict.items():
        if key != "Label":
            precision, recall, _ = precision_recall_curve(data_dict["Label"], value)
            pr_auc = auc(recall, precision)
            plt.plot(recall, precision, lw=lw, label=f'{key} (AUC = {pr_auc:.2f})')

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower left")
    plt.savefig('/media/concha-eloko/Linux/papers/Deposcope/Precision_Recall_methods.1303.png', dpi=500)
    plt.show()

plot_multiple_precision_recall_curves(data_dict)